In [1]:
!pip uninstall -y enum34

In [2]:
!pip install gym[atari]

You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [3]:
import torch
import torch.nn as nn
import random
import gym
import numpy as np
import random
import cv2
from tqdm import tqdm
import collections
import math

In [4]:
class ReplayBuffer():
    
    def __init__(self, capacity, state_shape):
        self.capacity = capacity
        
        self.states = torch.zeros(capacity, *state_shape)
        self.actions = torch.zeros(capacity, 1)
        self.rewards = torch.zeros(capacity, 1)
        self.next_states = torch.zeros(capacity, *state_shape)
        self.done = torch.zeros(capacity, 1)
        self.position = 0
        self.length = 0
        
        
    def push(self, state, action, reward, next_state, done):    
        self.states[self.position] = state
        self.actions[self.position] = action
        self.rewards[self.position] = reward
        self.next_states[self.position] = next_state
        self.done[self.position] = done
        
        self.position = (self.position + 1) % self.capacity
        self.length = min(self.length + 1, self.capacity)
        
    def sample(self, batch_size):
        indices = np.random.choice(range(self.length), size=batch_size)
        state_sample = self.states[indices]
        action_sample = self.actions[indices]
        reward_sample = self.rewards[indices]
        next_state_sample = self.next_states[indices]
        done_sample = self.done[indices]
        
        return state_sample, action_sample, reward_sample, next_state_sample, done_sample
        
    def __len__(self):
        return self.length

In [5]:
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        """Return only every `skip`-th frame"""
        super(MaxAndSkipEnv, self).__init__(env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        """Clear past frame buffer and init to first obs"""
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs


class ProcessFrame84(gym.ObservationWrapper):
    """
    Downsamples image to 84x84
    Greyscales image

    Returns numpy array
    """
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]),
                                                dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class ScaledFloatFrame(gym.ObservationWrapper):
    """Normalize pixel values in frame --> 0 to 1"""
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0


class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),
                                                old_space.high.repeat(n_steps, axis=0), dtype=dtype)

    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer


def make_env(env):
    env = MaxAndSkipEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4)
    env = ScaledFloatFrame(env)
    return env

In [11]:
class DQN(nn.Module):

    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)
    

class DoubleDQNAgent:
    
    def __init__(self, env, memory, policy_net, target_net, batch_size=32, lr=0.00025, target_update=5000,
                 gamma=0.9, exploration_max=0.9, exploration_min=0.02, exploration_steps=10000):
        # Env
        self.env = env
        self.state_shape = env.observation_space.shape
        self.action_space = env.action_space.n
        
        # Device used for training
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # Double DQN
        self.policy_net = policy_net.to(self.device)
        self.target_net = target_net.to(self.device)
        
        # DO NOT COMPUTE GRADIENTS FOR TARGET NET
        self.target_net.eval()
        
        # TODO optimizer choice
        self.lr = lr
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=self.lr)
        
        # Memory can be chose
        self.memory = memory
    
        # Learning parameters
        self.gamma = gamma
        self.loss_fn = nn.SmoothL1Loss().to(self.device) # Also known as Huber loss <-- TODO loss choice
        self.exploration_max = exploration_max
        self.exploration_rate = exploration_max
        self.exploration_min = exploration_min
        self.exploration_steps = exploration_steps
        self.target_update = target_update
        self.batch_size = batch_size
        
        # Counters and stats
        self.steps = 0
        self.episodes = 0
        self.reward_history = []
        self.loss_history = []
        
    def act(self, state):
        # Epsilon-greedy action
        self.steps += 1
        if random.random() < self.exploration_rate:  
            return torch.tensor([[random.randrange(self.action_space)]])
        # Local net is used for the policy
        return torch.argmax(self.policy_net(state.to(self.device))).unsqueeze(0).unsqueeze(0).cpu()
    
    def _update_target(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())
        
    def _update_exploration_rate(self):
        self.exploration_rate = self.exploration_min + (self.exploration_max - self.exploration_min) * math.exp(
            -1.0 * self.steps / self.exploration_steps
        )
    
    def optimize_model(self):
        
        # Update target DQN every `target_update` steps
        if self.steps % self.target_update == 0:
            self._update_target()
        
        # Replay buffer not full
        if self.batch_size > len(self.memory):
            return
        
        # Sample from buffer
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = self.memory.sample(self.batch_size)
        
        # Send to GPU
        state_batch = state_batch.to(self.device)
        action_batch = action_batch.to(self.device)
        reward_batch = reward_batch.to(self.device)
        next_state_batch = next_state_batch.to(self.device)
        done_batch = done_batch.to(self.device)
        
        # Zero optimizer gradients
        self.optimizer.zero_grad()
        
        # Compute target state-action values
        expected_state_action_values = reward_batch + torch.mul((self.gamma * self.target_net(next_state_batch).max(1).values.unsqueeze(1)), 1 - done_batch)
        
        # Compute state-action values from policy_net
        state_action_values = self.policy_net(state_batch).gather(1, action_batch.long())
        
        # Compute loss
        loss = self.loss_fn(state_action_values, expected_state_action_values)
        
        # Optimize model
        loss.backward()
        self.optimizer.step()
        
        # Decay exploration rate
        self._update_exploration_rate()
        
    def train(self, num_episodes):
        for episode in tqdm(range(num_episodes)):
            
            state = env.reset()
            state = torch.tensor(state)
            
            episode_reward = 0
            
            done = False
            while not done:
                
                self.steps += 1
                action = self.act(state.view((1,)+self.state_shape))
                
                next_state, reward, done, info = self.env.step(int(action[0]))
                
                next_state_tensor = torch.tensor(next_state)
                reward_tensor = torch.tensor(reward)
                done_tensor = torch.tensor(int(done)).unsqueeze(0)
                
                self.memory.push(state, action, reward_tensor, next_state_tensor, done_tensor)
                self.optimize_model()
                
                episode_reward += reward
                
                state = next_state_tensor
            
            self.reward_history.append(episode_reward)
            self.episodes += 1
            print("Episode reward for episode {} is {}".format(self.episodes + 1, episode_reward))
            print("Average reward is {}".format(np.mean(self.reward_history)))
            print("Epsilon is {}".format(np.mean(self.exploration_rate)))

            

In [7]:
env = gym.make('Breakout-v4')
env = make_env(env)

In [12]:
policy_net = DQN(env.observation_space.shape, env.action_space.n)
target_net = DQN(env.observation_space.shape, env.action_space.n)
memory = ReplayBuffer(30000, env.observation_space.shape)
agent = DoubleDQNAgent(env, memory, policy_net, target_net)

agent.train(1000)

  0%|          | 1/1000 [00:00<09:12,  1.81it/s]

Episode reward for episode 2 is 0.0
Average reward is 0.0
Epsilon is 0.8886340388178531


  0%|          | 2/1000 [00:01<11:35,  1.43it/s]

Episode reward for episode 3 is 1.0
Average reward is 0.5
Epsilon is 0.875017475126428


  0%|          | 3/1000 [00:02<12:26,  1.34it/s]

...

Episode reward for episode 432 is 8.0
Average reward is 4.6705336426914155
Epsilon is 0.02004525337198859


 43%|████▎     | 432/1000 [14:11<26:48,  2.83s/it]

Episode reward for episode 433 is 11.0
Average reward is 4.685185185185185
Epsilon is 0.020044153720209436


 43%|████▎     | 432/1000 [14:11<18:40,  1.97s/it]


KeyboardInterrupt: 